# 1. Import des librairies

In [2]:
import yfinance as yf
import ta
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.models import Model
from keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, Add, GlobalAveragePooling1D
from keras.losses import MeanSquaredError, MeanAbsoluteError
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import matplotlib.pyplot as plt

# 2. Chargement des données brutes (yfinance)

In [4]:
TRAIN_DATA_RATIO = 0.8
VALIDATION_DATA_RATIO = 0.2

NUMBER_OF_SERIES_FOR_PREDICTION = 24

In [5]:
gspc_data = yf.download('^GSPC', interval='5m', period='1mo')

[*********************100%***********************]  1 of 1 completed


In [7]:
# Ensure 'Close' is a Series
close_series = gspc_data['Close'].squeeze()

In [8]:
# Compute indicators
feature = pd.DataFrame(index=gspc_data.index)
feature['SMA'] = ta.trend.sma_indicator(close_series, window=14)
feature['MACD'] = ta.trend.macd(close_series)
feature['RSI'] = ta.momentum.rsi(close_series)
feature['Close'] = close_series

In [9]:
# Add features back to the original DataFrame
gspc_data['SMA'] = feature['SMA']
gspc_data['MACD'] = feature['MACD']
gspc_data['RSI'] = feature['RSI']

In [10]:
gspc_data

Price,Close,High,Low,Open,Volume,SMA,MACD,RSI
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,,,
Datetime,,,,,,,,
2024-12-26 14:30:00+00:00,6025.979980,6027.180176,6021.129883,6024.970215,0,NaN,NaN,NaN
2024-12-26 14:35:00+00:00,6025.339844,6029.200195,6022.930176,6026.140137,23990564,NaN,NaN,NaN
2024-12-26 14:40:00+00:00,6024.990234,6027.709961,6023.319824,6025.229980,19063176,NaN,NaN,NaN
2024-12-26 14:45:00+00:00,6022.910156,6027.959961,6022.450195,6024.870117,20310310,NaN,NaN,NaN
2024-12-26 14:50:00+00:00,6008.830078,6023.169922,6007.370117,6023.020020,23129377,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2025-01-24 20:35:00+00:00,6091.350098,6092.189941,6090.609863,6090.930176,31083000,6094.606445,-3.059816,38.482361
2025-01-24 20:40:00+00:00,6091.720215,6092.270020,6088.740234,6091.299805,34256000,6094.432896,-3.001261,39.379839


In [11]:
gspc_data.columns

MultiIndex([( 'Close', '^GSPC'),
            (  'High', '^GSPC'),
            (   'Low', '^GSPC'),
            (  'Open', '^GSPC'),
            ('Volume', '^GSPC'),
            (   'SMA',      ''),
            (  'MACD',      ''),
            (   'RSI',      '')],
           names=['Price', 'Ticker'])